In [75]:
import pandas as pd
from astropy.table import Table

In [76]:
field = "LOPS2"

In [77]:
# process targets
targets = pd.read_csv(f"../data/raw/plato_targets_{field}.csv")
targets = targets[["source_id", "flag", f"num_CCDs_{field}"]]

targets.rename(
    columns={
        "source_id": "sourceId",
        "flag": "Population",
    },
    inplace=True,
)
targets["Population"] = targets["Population"].map(
    {
        0: "Halo",
        1: "Thick Disk",
        2: "Thin Disk",
    }
)

In [78]:
# process asPIC

asPIC = Table.read(f"../data/external/asPIC_1.1.fits")
asPIC = asPIC[
    [
        "sourceId",
        "GLON",
        "GLAT",
        "gaiaV",
        "egaiaV",
        "Gmag",
        "eGmag",
        "Radius",
        "eRadius",
        "Mass",
        "eMass",
        "Teff",
        "eTeff",
    ]
]
for col in asPIC.colnames:
    asPIC[col] = asPIC[col][:, 0]
asPIC = asPIC.to_pandas()

In [79]:
# match asPIC and targets on sourceId
data = pd.merge(targets, asPIC, on="sourceId", how="inner")

In [80]:
# save
data.rename(columns={"sourceId": "gaia_sourceId"}, inplace=True)
data.to_csv(f"../data/processed/plato_targets_{field}_processed.csv", index=False)